In [6]:
import operator as op
def arith(a, b, operation):
    res = operation(a, b)
    return res
arith(1, 2, op.sub)

-1

In [7]:
from types import FunctionType

In [8]:
fstring = '''
def ar(a, b):
    op = __import__('operator')
    res = op.add(a, b)
    return res
    '''
print(fstring)


def ar(a, b):
    op = __import__('operator')
    res = op.add(a, b)
    return res
    


The compile() method computes the Python code from a source object and returns it.

In [9]:
# compile fstring into a code object
ftemplate = compile(fstring, 'fstring', 'exec')
ftemplate, ftemplate.co_consts[0]

(<code object <module> at 0x7f78e35d37c0, file "fstring", line 2>,
 <code object ar at 0x7f78e35d3030, file "fstring", line 2>)

In [15]:
type(ftemplate.co_consts[0])

code

In [17]:
help(type(ftemplate.co_consts[0]))

```
Help on class code in module builtins:

class code(object)
 |  code(argcount, posonlyargcount, kwonlyargcount, nlocals, stacksize,
 |        flags, codestring, constants, names, varnames, filename, name,
 |        firstlineno, lnotab[, freevars[, cellvars]])
 |  
 |  Create a code object.  Not for the faint of heart.
 ```

In [19]:
help(FunctionType)

```python
class function(object)
 |  function(code, globals, name=None, argdefs=None, closure=None)
 |  
 |  Create a function object.
 |  
 |  code
 |    a code object
 |  globals
 |    the globals dictionary
 |  name
 |    a string that overrides the name from the code object
 |  argdefs
 |    a tuple that specifies the default argument values
 |  closure
 |    a tuple that supplies the bindings for free variables
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __get__(self, instance, owner, /)
 |      Return an attribute of instance, which is of type owner.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 ```

In [15]:
# create a functiontype using ftemplate
dynamicfunction = FunctionType(ftemplate.co_consts[0], globals(), 'add')
dynamicfunction(1, 2)

3

In [2]:
a, b = 3, -5
def functiongenerator(fname, operator,):
    from types import FunctionType
    fstring = []
    # step 1: create the function blueprint as a string
    for i in operator:
        fstring.append('''
def arithmetic(a, b):
    op = __import__('operator')
    res = '''+ i +'''(a, b)
    return res
        ''')

    ftemplate = []
    for i in fstring:
        # step 2: actually compile the code and tell python that it needs to be exec
        ftemplate.append(compile(i, 'fstring', 'exec'))
    
    # step 3: assign FunctionType to the function using the code
    dynamicfunction = []
    for i,j in zip(ftemplate, fname):
        dynamicfunction.append(
            FunctionType(i.co_consts[0], globals(), j)
        )

    # actually access functions
    fdict = {}
    for i, j in zip(fname, dynamicfunction):
        fdict[i] = j

    for i in dynamicfunction:
        print(i(a, b))

    return fdict




In [4]:
operator = ['op.add','op.sub','op.mul','op.truediv','op.pow','op.mod', 'op.gt', 'op.lt']
fname = ['add','sub', 'multiply', 'divide', 'power','modulus', 'greaterthan', 'lesserthan']

functiongenerator(fname, operator)

-2
8
-15
-0.6
0.00411522633744856
-2
True
False


`globals` contains the global *symbol table*. Running `globals()` returns the `global` symbol table as a dictionary. 

Each key in the dictionary holds the name of the variable. This dictionary is also accessible from within functions and can be used to update global variables directly.

In [32]:
import pprint
import json
# print(json.dumps(globals() ))
g = globals()
pprint.pprint(g)

{'FunctionType': <class 'function'>,
 'In': ['',
        'def functiongenerator(fname, operator,):\n'
        '    from types import FunctionType\n'
        '    fstring = []\n'
        '    # step 1: create the function blueprint as a string\n'
        '    for i in operator:\n'
        "        fstring.append('''\n"
        'def arithmetic(a, b):\n'
        "    op = __import__('operator')\n"
        "    res = '''+ i +'''(a, b)\n"
        '    return res\n'
        "        ''')\n"
        '\n'
        '    ftemplate = []\n'
        '    for i in fstring:\n'
        '        # step 2: actually compile the code and tell python that it '
        'needs to be exec\n'
        "        ftemplate.append(compile(i, 'fstring', 'exec'))\n"
        '    \n'
        '    dynamicfunction = []\n'
        '    for i,j in zip(ftemplate, fname):\n'
        '        dynamicfunction.append(\n'
        '            FunctionType(i.co_consts[0], globals(), j)\n'
        '        )',
        'a, b = 3, -

In [33]:
from types import FunctionType
from pyspark.sql.functions import max

fstring = '''
def custom(col):
    importlib = __import__('importlib')
    F = importlib('pyspark.sql.functions') 
    return F.max(col)
'''

ftemplate = compile(fstring, 'custom', 'exec')
f = FunctionType(ftemplate.co_consts[0], globals(), 'custom')

In [31]:
import pandas as pd
from pyspark.sql import SparkSession

sc = SparkSession.builder.appName('imp').getOrCreate()
df = sc.createDataFrame(pd.DataFrame({
    'id': ['a', 'a', 'a'],
    'a': [1, 2, 3],
    'b': [4, 1, 0]
}))

In [35]:
f(1)

TypeError: 'module' object is not callable

In [30]:
import pyspark.sql.functions as F
df.withColumn('max_col', F.max('a'))

AnalysisException: grouping expressions sequence is empty, and 'id' is not an aggregate function. Wrap '(max(a) AS max_col)' in windowing function(s) or wrap 'id' in first() (or first_value) if you don't care which value you get.;
Aggregate [id#12, a#13L, b#14L, max(a#13L) AS max_col#19L]
+- LogicalRDD [id#12, a#13L, b#14L], false
